In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
# import pickle

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

In [2]:
## Load January data
df = pd.read_parquet('../notebooks/data/yellow_tripdata_2023-01.parquet')
df
# Q1: 19 columns

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.30,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.90,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.90,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.10,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.40,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066761,2,2023-01-31 23:58:34,2023-02-01 00:12:33,NaN,3.05,NaN,None,107,48,0,15.80,0.00,0.5,3.96,0.0,1.0,23.76,NaN,NaN
3066762,2,2023-01-31 23:31:09,2023-01-31 23:50:36,NaN,5.80,NaN,None,112,75,0,22.43,0.00,0.5,2.64,0.0,1.0,29.07,NaN,NaN
3066763,2,2023-01-31 23:01:05,2023-01-31 23:25:36,NaN,4.67,NaN,None,114,239,0,17.61,0.00,0.5,5.32,0.0,1.0,26.93,NaN,NaN
3066764,2,2023-01-31 23:40:00,2023-01-31 23:53:00,NaN,3.15,NaN,None,230,79,0,18.15,0.00,0.5,4.43,0.0,1.0,26.58,NaN,NaN


In [3]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)
df.duration.std()
# Q2: 42.59 minutes

42.59435124195458

In [4]:
((df.duration >= 1) & (df.duration <= 60)).mean()
# Q3: 98%

0.9812202822125979

In [5]:
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [6]:
categorical = ['PULocationID', 'DOLocationID']
df[categorical].dtypes


PULocationID    int64
DOLocationID    int64
dtype: object

In [7]:
df.loc[:,categorical] = df.loc[:,categorical].astype(str)
df[categorical].dtypes

/tmp/ipykernel_23418/491074066.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['161' '43' '48' ... '114' '230' '262']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:,categorical] = df.loc[:,categorical].astype(str)
/tmp/ipykernel_23418/491074066.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['141' '237' '238' ... '239' '79' '143']' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[:,categorical] = df.loc[:,categorical].astype(str)


PULocationID    object
DOLocationID    object
dtype: object

In [8]:
# df[categorical].iloc[:10].to_dict(orient='records')
tr_dict = df[categorical].to_dict(orient='records')

In [9]:
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(tr_dict)
X_train
# Q4: 515 columns

<3009173x515 sparse matrix of type '<class 'numpy.float64'>'
	with 6018346 stored elements in Compressed Sparse Row format>

In [10]:
target = 'duration'
y_train = df[target].values
# sns.histplot(y_train)

In [11]:
## train using January data
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)
# sns.histplot(y_pred)
root_mean_squared_error(y_train, y_pred)
# Q5: RMSE = 7.649 minutes

7.649261932106969

In [12]:
## validate using February data
dg = pd.read_parquet('../notebooks/data/yellow_tripdata_2023-02.parquet')
dg['duration'] = dg.tpep_dropoff_datetime - dg.tpep_pickup_datetime
dg.duration = dg.duration.apply(lambda td: td.total_seconds() / 60)
dg = dg[(dg.duration >= 1) & (dg.duration <= 60)]


In [13]:
# pd.options.display.float_format = '{:,.0f}'.format
dg[categorical] = dg[categorical].round(0).astype(int)
# dg[categorical]


In [14]:
dg[categorical] = dg[categorical].astype(str)
# dg[categorical].dtypes
val_dict = dg[categorical].to_dict(orient='records')
val_dict

[{'PULocationID': '142', 'DOLocationID': '163'},
 {'PULocationID': '132', 'DOLocationID': '26'},
 {'PULocationID': '161', 'DOLocationID': '145'},
 {'PULocationID': '148', 'DOLocationID': '236'},
 {'PULocationID': '137', 'DOLocationID': '244'},
 {'PULocationID': '263', 'DOLocationID': '141'},
 {'PULocationID': '48', 'DOLocationID': '243'},
 {'PULocationID': '114', 'DOLocationID': '211'},
 {'PULocationID': '114', 'DOLocationID': '249'},
 {'PULocationID': '125', 'DOLocationID': '107'},
 {'PULocationID': '140', 'DOLocationID': '42'},
 {'PULocationID': '140', 'DOLocationID': '226'},
 {'PULocationID': '249', 'DOLocationID': '90'},
 {'PULocationID': '234', 'DOLocationID': '4'},
 {'PULocationID': '114', 'DOLocationID': '125'},
 {'PULocationID': '132', 'DOLocationID': '239'},
 {'PULocationID': '132', 'DOLocationID': '230'},
 {'PULocationID': '140', 'DOLocationID': '68'},
 {'PULocationID': '144', 'DOLocationID': '79'},
 {'PULocationID': '132', 'DOLocationID': '90'},
 {'PULocationID': '236', 'DOL

In [15]:
X_val = dv.transform(val_dict)
X_val

<2855951x515 sparse matrix of type '<class 'numpy.float64'>'
	with 5711894 stored elements in Compressed Sparse Row format>

In [16]:
target = 'duration'
y_val = dg[target].values
# sns.histplot(y_val)


In [17]:
y_pred_val = lr.predict(X_val)
root_mean_squared_error(y_val, y_pred_val)
# Q6: RMSE in validation model = 13.32 minutes

7.811818743246608